# 第６章　k平均法：教師なし学習モデルの基礎
---
教師なし学習によるクラスタリングの基礎として、「k平均法」のアルゴリズムを解説する。
具体的な応用例として、画像ファイルの減色処理の話があるが、概要を説明して、詳細は割愛。
文書データの自動分類など、単純ながらも応用範囲の広いアルゴリズム。
さらに、怠惰学習モデルであり、k平均法に類似のアルゴリズムであるk近傍法を紹介。
> keyword: **k平均法**, **k近傍法**

* クラスタリング ：目的変数のない（教師なしの）場合
* 分類アルゴリズム  ：目的変数がある（教師ありの）場合 

## 6.1 k平均法によるクラスタリングと応用例
### 6.1.1 教師なし学習モデルとしてのクラスタリング
k平均法は「教師なし学習」と呼ばれる手法。
6章までに出てきたアルゴリズムは全て「教師あり学習」。
* 教師あり学習：当たれられるデータは変数 $t_n$ で表される値(=目的変数：新たなデータに対して推定を行う対象となるもの)をもっていた。
* 教師なし学習：分析対象のデータには目的変数は含まれていない。

### 6.1.2 k平均法によるクラスタリング
![図6.1](pic6.1.png)
トレーニングセットとして、$(x,y)$平面上の多数の点  $\{(x_n,y_n)\}_{n=1}^N$ が与えられたとする。トレーニングセットのデータには、目的関数 $t_n$ は含まれてない。k平均法を用いて、これらのデータを2つのクラスターに分類(分類してできる各グループのこと)。分類するクラスターの数は事前に指定する必要がある。今回の事例ではクラスター数＝２。$(x,y)$ 平面上の点はベクトル記号で表す。トレーニングセットに含まれる点は $ \bf{x}$$_n +(x_n,y_n)^T$と表記。まず、クラスターの代表点を容易。クラスター数が2つなので、2点を $\{{\mu}_k\}_{k=1}^2$ を代表点として設定。トレーニングセットの各点について、どちらの代表点に所属するかを決定する。代表点との距離 $\|\bf{x}$$_n-{\mu}_k\|$ を計算して、距離が短い方の代表点に所属するものと決定。さらに、それぞれの点のどちらの代表点に所属するかを示す変数$r_{nk}$を定義しておく。\n",

\\begin{equation}
r_{nk} = \begin{cases}
1 & (\bf{x} _n がk番目の代表点に属する場合) \\
0 & (otherwise)
\end{cases}
\\end{equation}
    
この分類は、最初に決めた代表点の取り方に依存するので、必ずしも最適な分類とは限らない。そこで、現在のクラスターを元にして、あらためて代表点を取り直す。具体的にはそれぞれのクラスターに所属する点の重心を新たな代表点とする。(例えば、3つの点の重心は3点の各成分の合計を3で割った値)

\\begin{equation}
{\mu}_k = \frac{\Sigma \bf{x}_n}{N_k}    (k=1,2)
\\end{equation}
 
ここで$N_k$は$k$番目の代表点に所属する点の個数で、分子の$\Sigma$は$k$番目の代表点に所属する点についてのみ加えている。$r_{nk}$で表すと以下のとおり

${\mu}_k = \dfrac{\sum_{n=1}^N r_{nk} \bf{x}_n}{\sum_{n=1}^N r_{nk}}$

各クラスターの重心として、新たに決まった代表点を示してる(d)。この新たな代表点を元にして、再度、トレーニングセットの各点がどちらの代表点に所属するかを決め直す。すると、(e)のような適切な分類が行われる。この後に、同じ手続きを繰り返し、ある条件において変化量あるいは変化率が一定値より小さくなれば終了(f)。最終的に得られた$\{{\mu}_k\}_{k=1}^2$ が代表点となる。複雑なトレーニングセット、より多数のクラスターに分類する場合は、最初の代表点の取り方によって結果がかわることもある。k平均法を現実の問題に適用する際は、最初の代表点の取り方を変えながら、何度か計算を繰り返して、より適切と思われるクラスターを発見するなどの工夫が必要。

### 6.1.3 画像データへの応用
ベクトルは3次元、クラスター数2,3,5,16で計算している。
> keyword: **二乗歪み**

二乗歪みの値の変化(減少分)が0.1%以下になったところで計算を打ち切り、その時点の代表点を最終的な答えとしている。

### 6.1.4 サンプルコードによる確認

In [ ]:
# -*- coding: utf-8 -*-
#
# k平均法による画像の減色処理
#
# 2015/04/24 ver1.0
#

import numpy as np
from numpy.random import randint
from PIL import Image

#------------#
# Parameters #
#------------#
Colors = [2, 3, 5, 16]  # 減色後の色数（任意の個数の色数を指定できます）


# k平均法による減色処理
def run_kmeans(pixels, k):
    cls = [0] * len(pixels)

    # 代表色の初期値をランダムに設定
    center = []
    for i in range(k):
        center.append(np.array([randint(256), randint(256), randint(256)]))
    print "Initial centers:",
    print map(lambda x: x.tolist(), center)
    print "========================"
    distortion = 0.0

    # 最大50回のIterationを実施
    for iter_num in range(50): 
        center_new = []
        for i in range(k):
            center_new.append(np.array([0,0,0]))
        num_points = [0] * k
        distortion_new = 0.0

        # E Phase: 各データが属するグループ（代表色）を計算
        for pix, point in enumerate(pixels):
            min_dist = 256*256*3
            point = np.array(point)
            for i in range(k):
                d = sum([x*x for x in point-center[i]])
                if d < min_dist:
                    min_dist = d
                    cls[pix] = i
            center_new[cls[pix]] += point
            num_points[cls[pix]] += 1
            distortion_new += min_dist

        # M Phase: 新しい代表色を計算
        for i in range(k):
            center_new[i] = center_new[i] / num_points[i]
        center = center_new
        print map(lambda x: x.tolist(), center)
        print "Distortion: J=%d" % distortion_new

        # Distortion(J)の変化が0.1%未満になったら終了
        if iter_num > 0 and distortion - distortion_new < distortion * 0.001:
            break
        distortion = distortion_new

    # 画像データの各ピクセルを代表色で置き換え
    for pix, point in enumerate(pixels):
        pixels[pix] = tuple(center[cls[pix]])

    return pixels
        
# Main
if __name__ == '__main__':
    for k in Colors:
        print ""
        print "========================"
        print "Number of clusters: K=%d" % k
        # 画像ファイルの読み込み
        im = Image.open("photo.jpg")
        pixels = list(im.convert('RGB').getdata())
        # k平均法による減色処理
        result = run_kmeans(pixels, k)
        # 画像データの更新とファイル出力
        im.putdata(result) # Update image
        im.save("output%02d.bmp" % k, "BMP")

### 6.1.5 k平均法の数学的根拠
k平均法は、数学的には特定のグループ分けに対する「歪み」の値を計算して、歪みがなるべき小さくなるグループ分けを探していく手法。

二乗歪み：
$J= \sum_{n=1}^N \sum_{k=1}^K r_{nk} \|\bf{x}$$_n-{\mu}_k\|^2$
これは、各データについての、「自身が所属するクラスターの代表点からの距離の2乗」を合計した値。
$J$の値を小さくするということは、代表点のなるべく近くにデータが集まるように分類するということ。

> 数学徒の小部屋

各データが所属するクラスターを次の記号で表す。

\\begin{equation}
r_{nk} = \begin{cases}
1 & (\bf{x} _n がk番目の代表点に属する場合) \\
0 & (otherwise)
\end{cases}
\\end{equation}

現在の分類状態における「二乗歪み」を以下のように定義：

$J= \sum_{n=1}^N \sum_{k=1}^K r_{nk} \|\bf{x}_n-{\mu}_k\|^2$

各データについての「自身が所属するクラスターの代表点からの距離の2乗」を合計した値。
このあと、k平均法の手続きにしたがって、$r_{nk}$と${\mu}_k$を修正していくと、$J$の値は減少。
最終的に極小値に達することを示すことになる。

各データが所属するクラスターを選択しなおす。この時、各データ$\bf{x}_n$について、代表点からの距離$\|\bf{x}_n-{\mu}_k\|$が最も小さいクラスターを選択する。この操作によって、$J$の値が大きくなることはない。このとき、$r_{nk}$の値を下記の条件で、再定義できる。

\\begin{equation}
r_{nk} = \begin{cases}
1 & (k=  argmin_{k'}  \|\bf{x}_n-{\mu}_k\|) の場合\\
0 & (otherwise)
\end{cases}
\\end{equation}
※$argmin_{k'}  f_{k'}$は$f_{k'}$を最小にする$k'$

$J$の値を最小にするという条件で、${\mu}_k$を選択してみる。＄J$の式は＄{\mu}_k＄でみると、下に凸の二次関数。偏微分係数が$0$になるという条件で最小化することが可能。
$J$を成分表記すると以下のとおり：

$J= \sum_{n=1}^N \sum_{k=1}^K \{ r_{nk} \sum_{i}([\bf{x}_n]_i-[{\mu}_k]_i)^2$

特定成分における偏微分係数は以下のとおり：

$\dfrac{\partial J}{\partial [{\mu}_k]_i}=-2 \sum_{n=1}^N  \{ r_{nk} ([\bf{x}_n]_i-[{\mu}_k]_i)$

これが$0$になる。式変形すると以下のとおり：

$[{\mu}_k]_i = \dfrac{\sum_{n=1}^N r_{nk} [\bf{x}_n]_i}{\sum_{n=1}^N r_{nk}}$

成分表記をベクトルになおすと、以下のとおり：

${\mu}_k = \dfrac{\sum_{n=1}^N r_{nk} \bf{x}_n}{\sum_{n=1}^N r_{nk}}$

これは、各クラスターの重心を新たな代表点にとるという6.1.2の式の手続きと同じで、$J$がおおきくなることはない。
つまり、$J$の値は必ず小さくなるか、もしくは変化しない極小値に達する。
各クラスターの代表点はデータの分類によって一意にきまるので、$J$の値はデータの分類方法で決まる。したがって、$J$がとり得る値の個数は高々$N$個のデータを$K$個の組にわける場合の数($=_NC_K$)であり、有限個になる。したがって、$J$の値が無限に減少を続けることはなく、有限回の操作で必ず極小値に達する。

## 6.2 怠惰学習モデルとしてのk近傍法
k近傍法は教師あり学習の分類アルゴリズム。

### 6.2.1 k近傍法による分類
トレーニングセットのデータは目的変数$t_n$の値が付与される。
本勉強会では確認してない章であるが、1.3.2 線形判別による新規データの分類の例題2と同じトレーニングセット$\{(x_n,y_n,t_n)\}_{n=1}^N$を考える。

パーセプトロンやロジスティック回帰では、目的変数の値は２種類あり、その２種類の属性データを$(x,y)$平面上の直線で分類することを考えた。
未知のパラメータ$\bf{w}$を含む形で、直線の方程式を用意して、機械学習によって、パラメータの値を決定した。

ｋ近傍法では、$\bf{w}$のようなパラメータは登場しない。機械学習で決定することもない。

新たなデータ$(x,y)$が与えられた際にその周りのデータをみて、自分の近くにあるデータの目的変数の値から、自分自信の目的変数を推定するということを行う。

最も単純な例は、一番近くにあるデータと同じ属性、つまり「目的変数の値」をもっているものと推定。一般的に、自分の周りの$K$個分のデータ(近い方から$K$個分のデータ)を見て、その中で最も個数が多い目的変数の値を採用。すなわち、自分の周りの$K$個のデータによる「多数決」で決定する。

![図6.8](pic6.8.png)
図6.8は$(x,y)$平面上にランダムに生成した2種類の属性のデータ群について$K=1$と$K=3$でk近傍法による分類を実施した結果。

$K=1$の場合は、単独で存在するデータの周りに離れ小島ができている。$K=3$の場合は、3個分のデータから判断するため、単独で存在するデータは「多数決」にまけて、離れ小島がなくなる。$K=3$のほうが自然な分類。


### 6.2.2 k近傍法の問題点
１つは、新たなデータの分類を判定するのにかかる時間。その都度、トレーニングセットに含まれる全てのデータを参照して、自分に近いデータを探しだす必要があるから。
もう1つは、分析の「モデル」が明確ではない。仮設がない。与えられたデータをみて、そこからわかる事実を元に判断をしているだけ。なぜうまくいったのかがわからない。

現実問題、１つのアルゴリズムを試して、それだけでうまくいくことはない。様々な仮設を立てて、それぞれについて、うまくいく理由やうまくいかない理由を考える必要がある。